In [29]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit import BasicAer
from scipy.optimize import minimize
from qiskit.quantum_info import Statevector

In [34]:
def hourglass_solver(profits, weights, probs, max_weight, p_total):
    init_s = [2 * np.arccos(np.sqrt(1 - p)) for p in probs]
    
    def expectation_value(params):
        p = len(params) // 2
        beta_s = params[:p]
        gamma_s = params[p:]
        n_qubits = len(profits)
        p_total = len(beta_s)
        qubits = QuantumRegister(n_qubits)
        bits = ClassicalRegister(n_qubits)
        circuit = QuantumCircuit(qubits, bits)
        
        # Qubits initialisieren 
        for i in range(n_qubits):
            
            circuit.p(init_s[i], qubits[i])

        # QAOA anwenden    
        for p in range(p_total):
            for i in range(n_qubits):
                circuit.rz(gamma_s[p] * weights[i] * 2, qubits[i])
            for i in range(n_qubits):
                phi = 2 * np.arcsin(np.sqrt(probs[i]))
                circuit.ry(phi, qubits[i]).inverse()
                circuit.rz(2 * beta_s[p], qubits[i])
                circuit.ry(phi, qubits[i])

        test_state = Statevector.from_int(0, 2 ** n_qubits)
        print(test_state.evolve(circuit).draw())

        # Finalen Zustand messen
        for i in range(n_qubits):
            circuit.measure(qubits[i], bits[i])
        backend = BasicAer.get_backend("qasm_simulator")
        job = backend.run(transpile(circuit, backend), shots=100)
        results = job.result().get_counts(circuit)
        outcomes = list(process_outcomes(results, profits, weights, max_weight))
        res = min(outcomes)
        return res


    params = [1.0 for _ in range(2 * p_total)]

    res = minimize(expectation_value, params, method="COBYLA")
    
    return res

In [35]:
def process_outcomes(result_dict, profits, weights, max_weight):
    def bitstring_to_bitint(bitstring):
            return np.array([int(c) for c in bitstring])
    outcome_bits = [bitstring_to_bitint(o) for o in list(result_dict.keys())]
    outcomes = -1 * np.array([(o * np.array(profits)).sum() * int((o * np.array(weights)).sum() <= max_weight) for o in outcome_bits])
    return outcomes

In [37]:
n_qubits = 3
profits = [i for i in range(n_qubits)]
weights = [i for i in range(n_qubits)]
max_weight = sum(weights)
p_total = 1
probs = [0.5 for _ in range(n_qubits)]

hourglass_solver(profits, weights, probs, max_weight, p_total)

Statevector([ 0.08408258-0.58986053j,  0.37874509+0.0539888j ,
              0.37874509+0.0539888j , -0.03466581+0.24318943j,
              0.37874509+0.0539888j , -0.03466581+0.24318943j,
             -0.03466581+0.24318943j, -0.15615014-0.02225866j],
            dims=(2, 2, 2))
Statevector([ 0.10609782-0.74430303j, -0.34063591-0.04855647j,
             -0.34063591-0.04855647j, -0.02222224+0.1558946j ,
             -0.34063591-0.04855647j, -0.02222224+0.1558946j ,
             -0.02222224+0.1558946j ,  0.07134634+0.01017017j],
            dims=(2, 2, 2))
Statevector([-0.16648225+0.57209177j, -0.3673359 -0.10689702j,
             -0.3673359 -0.10689702j,  0.06863779-0.23586367j,
             -0.3673359 -0.10689702j,  0.06863779-0.23586367j,
              0.06863779-0.23586367j,  0.15144632+0.04407182j],
            dims=(2, 2, 2))
Statevector([ 0.10365605-0.7271734j ,  0.34750233+0.04953526j,
              0.34750233+0.04953526j, -0.02367196+0.16606475j,
              0.34750233+0.0495

     fun: -3.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 13
  status: 1
 success: True
       x: array([1., 1.])